# Mapping Open Innovation Labs Residencies

This is an experiment to learn [Leaflet](http://leafletjs.com/) embedded in Jupyter via the [folium](https://github.com/python-visualization/folium). It makes use of [GeoJSON](http://leafletjs.com/examples/geojson/). Future versions of this experiment will serve the GeoJSON from [GeoServer](http://geoserver.org/). 

I'm making extensive use of the [examples](https://github.com/python-visualization/folium/tree/master/examples)


## Known Issues

none so far, looks solid


### First, Download the Data

This will move to GeoServer in the future

In [4]:
%%bash
curl -O https://raw.githubusercontent.com/sherl0cks/labs-big-data/master/notebooks/data/residencies.geo.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5412  100  5412    0     0  35341      0 --:--:-- --:--:-- --:--:-- 35605


### Then Create The Map

This is making use of GeoJSON's flexible `properties` to provide interesting data about a residency, and then translate that info, plus the coordinates, to [Leaflet Markers](http://leafletjs.com/examples/quick-start/).

In [5]:
import folium
import json
import os

from folium.plugins import MarkerCluster


# could generate html here which would allow for new lines
def create_popup_text(feature):
    popup = f['properties']['customer']
    popup += "\n\n\n"
    popup += f['properties']['residencyType'] 
    popup += ' residency at a '
    popup += f['properties']['facilityType']
    popup += ' facility'
    if 'residencyFocus' in f['properties']:
        popup += ' focused on '
        popup += f['properties']['residencyFocus']
    if 'technologies' in f['properties']:
        popup += ' using '
        popup += f['properties']['technologies']
    return popup

# This is more along the lines of the school mapping use case
def create_circle_markers(feature):
    return folium.CircleMarker(
        location=f['geometry']['coordinates'], 
        popup=folium.Popup(html=create_popup_text(f)),
        fill=True,
        radius=8
    )
 
# This looks better for our use case
def create_markers(feature):
    return folium.Marker(
        location=f['geometry']['coordinates'], 
        popup=folium.Popup(html=create_popup_text(f))
    )


m = folium.Map(
    location=[0, 0],
    zoom_start=2,
    # https://github.com/ellisonbg/ipyleaflet/blob/master/ipyleaflet/leaflet.py#L104
    tiles='http://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
    attr='Tiles &copy; Esri &mdash; Source: Esri, i-cubed, USDA, USGS, AEX, GeoEye, Getmapping, Aerogrid, IGN, IGP, UPR-EGP, and the GIS User Community'
)

marker_cluster = MarkerCluster().add_to(m)

with open('./residencies.geo.json') as f:
    data = json.load(f)

for f in data['features']:
    create_circle_markers(f).add_to(marker_cluster)

m